# 00 Setup

## Create VIEW from system.billing

In [0]:
%sql
CREATE OR REPLACE VIEW main.billing.view_usage
COMMENT 'The usage table gives you access to account-wide billable usage data. The data is centralized and routed to all regions.' AS
(
  SELECT
    workspace_id,
    sku_name,
    usage_start_time,
    usage_end_time,
    usage_date,
    usage_unit,
    usage_quantity,
    billing_origin_product,
    usage_type
  FROM
    system.billing.usage
)

In [0]:
%sql
CREATE OR REPLACE VIEW main.billing.view_list_prices (
  price_start_time, price_end_time, sku_name, currency_code, usage_unit, pricing
  COMMENT 'The single price that can be used for simple long-term estimates'
)
COMMENT 'The pricing table gives you access to a historical log of SKU pricing. A record gets added each time there is a change to a SKU price.' AS
(
  SELECT
    price_start_time,
    price_end_time,
    sku_name,
    currency_code,
    usage_unit,
    pricing.effective_list.default AS pricing
  FROM
    system.billing.list_prices
)

## RI

In [0]:
%sql
CREATE OR REPLACE TABLE main.billing.azure_dbu_discount(
  dbu DOUBLE COMMENT 'The purchase volume of DBU is denominated in USD',
  discount DOUBLE COMMENT 'The discount rate'
)
COMMENT 'The discount table records the discounts available for each purchase volume when reserving instances (RI) of Azure Databricks DBU.';

INSERT INTO main.billing.azure_dbu_discount (dbu, discount)
  VALUES
    (0, 0),
    (12500, 0.04),
    (25000, 0.06),
    (50000, 0.08),
    (100000, 0.11),
    (200000, 0.14),
    (350000, 0.18),
    (500000, 0.20),
    (750000, 0.23),
    (1000000, 0.27),
    (1500000, 0.30),
    (2000000, 0.33);

In [0]:
%sql
CREATE OR REPLACE FUNCTION main.billing.get_dbu_discount()
  RETURNS TABLE(dbu DOUBLE, discount DOUBLE)
  COMMENT 'Get information about the available discounts for each purchase volume when reserving Azure Databricks DBU Reserved Instances (RI).'
  RETURN
    SELECT
      *
    FROM
      main.billing.azure_dbu_discount

## Genie

Based on [the document](https://docs.databricks.com/aws/en/genie/set-up), establish Genie Space.